Allows the notebook to load your username and password from the .env file.

In [34]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


### Import dependancies and connect to peloton.

You should see a 200 status if it's connected successfully.

In [35]:
import requests
import pandas as pd
import json
import os
import plotly.express as px
from datetime import datetime

s = requests.Session()
payload = {'username_or_email': os.getenv("peloton-username"), 'password': os.getenv("peloton-password")}
s.post('https://api.onepeloton.com/auth/login', json=payload)

<Response [200]>

## Load user data from api

In [36]:
url = 'https://api.onepeloton.com/api/me'
# getting data from source
data = s.get(url).json()

pretty_data = json.dumps(data, indent=4)

print(pretty_data)

{
    "email": "james_wooltorton@hotmail.com",
    "referrals_made": 0,
    "facebook_id": null,
    "can_charge": true,
    "created_at": 1589042180,
    "name": "Jamie Wooltorton",
    "weight": 231.49,
    "cycling_workout_ftp": 0,
    "is_profile_private": false,
    "is_provisional": false,
    "instructor_id": null,
    "total_pending_followers": 0,
    "phone_number": "7752332161",
    "is_complete_profile": true,
    "has_active_digital_subscription": false,
    "member_groups": [],
    "last_workout_at": 1601483700,
    "quick_hits": {
        "quick_hits_enabled": true,
        "speed_shortcuts": null,
        "incline_shortcuts": null
    },
    "total_workouts": 64,
    "is_demo": false,
    "cycling_ftp_source": null,
    "total_following": 0,
    "default_heart_rate_zones": [
        0,
        113.75,
        131.25,
        148.75,
        166.25
    ],
    "is_strava_authenticated": true,
    "paired_devices": [
        {
            "name": "AKG N60NC Wireless",
     

Show your peloton_id.

In [37]:
peloton_id = data['id']
print(peloton_id)

39cb085b05254d63a689a1c4324c65f8


Build the core DataFrame from workoutdata.

In [38]:
df = pd.DataFrame(columns=["percentile", 
                            "workout_date", 
                            "difficulty_rating_avg", 
                            "duration_mins", 
                            "class_length",
                            "total_work"])

races = 50


def get_workout(workout_id: str):
    url = 'https://api.onepeloton.com/api/workout/' + workout_id
    # getting data from source
    data = s.get(url).json()
    
    leaderboard_rank = data['leaderboard_rank']
    total_leaderboard_users= data['total_leaderboard_users']
    avg_pos = (leaderboard_rank / total_leaderboard_users)
    timestamp = datetime.fromtimestamp(data['created_at'])
    difficulty_rating_avg = data['ride']['difficulty_rating_avg']
    pdd = (data['ride']['pedaling_duration'] / 60)
    total_work = data['total_work']/ 1000
    new_row = pd.Series({"percentile": avg_pos, 
                         "workout_date": timestamp, 
                         "difficulty_rating_avg": difficulty_rating_avg, 
                         "duration_mins": pdd, 
                         "class_length": '{0} mins'.format(int(pdd)),
                         "total_work": total_work})
    global df
    df = df.append(new_row, ignore_index=True)


def get_session_data(session_type: str):
    for w in get_data.json()['data']:
        if w['ride']['difficulty_level'] !=  "beginner":
            if w['fitness_discipline'] == session_type:
                get_workout(w['id'])
 

url = 'https://api.onepeloton.com/api/user/{0}/workouts/?joins=ride,ride.instructor&limit={1}&page=0'.format(peloton_id, races)
# getting data from source
get_data = s.get(url)
get_session_data('cycling')


df['workout_date'] = pd.to_datetime(df['workout_date'])

In [39]:
print(df.head())

   percentile        workout_date  difficulty_rating_avg  duration_mins  \
0    0.717616 2020-09-30 17:35:00                 8.2566           45.0   
1    0.462977 2020-09-28 17:19:12                 8.1941           30.0   
2    0.604837 2020-09-25 17:04:33                 9.0351           30.0   
3    0.588778 2020-09-23 17:50:28                 8.5495           30.0   
4    0.377758 2020-09-21 17:19:49                 8.1728           20.0   

  class_length  total_work  
0      45 mins   280.87232  
1      30 mins   234.16660  
2      30 mins   234.72809  
3      30 mins   215.25122  
4      20 mins   162.37018  


## Build a scatter plot

In [40]:
def plot_workout_percentile():
    fig1 = px.scatter(df, x="workout_date", 
                         y="percentile", 
                         title='Workout % position',  
                         trendline="ols", 
                         trendline_color_override="red",
                    )
    fig1.update_yaxes(range=[0, 1])
    fig1.show()

plot_workout_percentile()